In [1]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import statsmodels.formula.api as smf

In [2]:
election_results = '../../data/Brazil/IBGE/series_pt.csv'
pib = '../../data/Brazil/IBGE/PIB/PIB_MUNICIPIOS_2010_2017.csv'
reference = '../../data/Brazil/IBGE/tse-ibge-correspondence.csv'

In [3]:
election_results = pd.read_csv(election_results, encoding='utf-8')
pib = pd.read_csv(pib, sep=';', encoding='latin')
reference = pd.read_csv(reference)
pib.head()

,ANO,CD_GRANDE_REGIAO,NM_GRANDE_REGIAO,CD_UF,SG_UF,NM_UF,CD_MUNICIPIO,NM_MUNICIPIO,REGIAO_METROPOLITANA,CD_MESORREGIAO,...,VAB_INDUSTRIA,VAB_SERVICOS,VAB_ADMINISTRACAO,VAB_TOTAL,IMPOSTOS,PIB,PIB_PER_CAPITA,ATIVIDADE_SEGUNDO_MAIOR_ADICIONADO_BRUTO,ATIVIDADE_MAIOR_ADICIONADO_BRUTO,ATIVIDADE_TERCEIRO_MAIOR_ADICIONADO_BRUTO
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,...,16.119,62.496,93.245,241.120,20.957,262.077,"10.731,18","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,...,287.139,494.946,343.868,1.199.664,165.030,1.364.694,"15.103,86","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,...,3.253,12.677,25.170,65.401,4.210,69.611,"11.033,62","Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,...,182.052,465.447,298.454,1.041.212,145.282,1.186.494,"15.095,15","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,...,19.734,80.725,63.018,192.454,29.567,222.021,"13.037,06","Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...


In [4]:
year = 2010
pib = pib[pib.ANO == year]
col_votes = 'percentual_votos_{}'.format(year)

In [5]:
reference['id'] = reference['UF'] + reference['COD_TSE'].astype(str)
reference['CD_MUNICIPIO'] = reference['CD_GEOCMU']
reference.drop(['CD_GEOCMU', 'AJUSTE'], axis=1, inplace=True)
#election_results['id'] = election_results['SG_ UF'] + election_results['CD_MUNICIPIO'].astype(str)
election_results['COD_TSE'] = election_results['cod_mun'].astype('int64')
data = reference.merge(pib[['CD_MUNICIPIO','PIB','PIB_PER_CAPITA']], on='CD_MUNICIPIO', how='left')
data = data.merge(election_results, on='COD_TSE', how='left')

In [6]:
data.head()

,chave,NOME,UF,COD_TSE,id,CD_MUNICIPIO,PIB,PIB_PER_CAPITA,cod_mun,percentual_votos_1994,percentual_votos_1998,percentual_votos_2002,percentual_votos_2006,percentual_votos_2010,percentual_votos_2014,percentual_votos_2018
0,ac acrelandia,ACRELÂNDIA,AC,1120,AC1120,1200013,121.915,"9.723,62",1120.0,0.196668,0.188771,0.437648,0.360062,0.219681,0.350076,0.163887
1,ac assis brasil,ASSIS BRASIL,AC,1570,AC1570,1200054,42.017,"6.916,45",1570.0,0.266833,0.305432,0.543697,0.518658,0.384841,0.475006,0.417841
2,ac brasileia,BRASILÉIA,AC,1058,AC1058,1200104,190.699,"8.895,37",1058.0,0.236119,0.274491,0.512855,0.455038,0.268854,0.360175,0.257246
3,ac bujari,BUJARI,AC,1007,AC1007,1200138,86.265,"10.179,96",1007.0,0.168473,0.245450,0.447928,0.390629,0.235370,0.293810,0.172761
4,ac capixaba,CAPIXABA,AC,1015,AC1015,1200179,108.531,"12.319,08",1015.0,0.144309,0.198480,0.465089,0.344324,0.187897,0.248154,0.177523


In [7]:
coef = 'PIB'
data['popularity'] = [.38]*len(data)
data[coef] = data[coef].str.replace('.', '')
data[coef] = data[coef].str.replace(',','.')
data[coef] = data[coef].astype('float64')
data[coef]=(data[coef]-data[coef].min())/(data[coef].max()-data[coef].min())
data[data.isnull().any(axis=1)]
data.dropna(inplace=True)
print(len(data))

5565


In [8]:
X = data[[coef,'popularity']]
Y = data[col_votes]
regr = linear_model.LinearRegression()
regr.fit(X,Y)

# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)
#Predict
ypred = model.predict(X)
# calc rmse
value_rmse = rmse(Y, ypred)
print('RMSE: {}'.format(value_rmse))

                              OLS Regression Results                             
Dep. Variable:     percentual_votos_2010   R-squared:                       0.006
Model:                               OLS   Adj. R-squared:                  0.006
Method:                    Least Squares   F-statistic:                     33.89
Date:                   Mon, 13 Jul 2020   Prob (F-statistic):           6.17e-09
Time:                           15:58:49   Log-Likelihood:                 2381.4
No. Observations:                   5565   AIC:                            -4759.
Df Residuals:                       5563   BIC:                            -4745.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PIB           -0.7498 

In [9]:
formula = col_votes + ' ~ ' + coef + '+ popularity'
md = smf.mixedlm(formula, data, groups=data["UF"])
mdf = md.fit()
print(mdf.summary())
#Predict
ypred = mdf.predict(X)
# calc rmse
value_rmse = rmse(Y, ypred)
print('RMSE: {}'.format(value_rmse))

C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2112: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(


               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: percentual_votos_2010
No. Observations: 5565    Method:             REML                 
No. Groups:       27      Scale:              0.0105               
Min. group size:  1       Log-Likelihood:     4714.8318            
Max. group size:  853     Converged:          Yes                  
Mean group size:  206.1                                            
---------------------------------------------------------------------
                Coef.     Std.Err.     z      P>|z|   [0.025   0.975]
---------------------------------------------------------------------
Intercept        43.300                                              
PIB              -0.333      0.086   -3.862   0.000   -0.502   -0.164
popularity     -112.502                                              
Group Var         0.018                                              

RMSE: 0.15795035951428715


C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2170: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2620: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))
C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
C:\Users\Tiago\Anaconda3\envs\Election_Forensics\lib\site-packages\statsmodels\base\model.py:1354: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
